In [ ]:
#si-exercise

import pandas as pd
import numpy as np
import scipy as sp

class RegressionModel:
    def __init__(self, x: pd.DataFrame, y: pd.DataFrame, create_intercept: bool, regression_type="ols"):
        self.x = x.copy()  # Store the exogenous variables
        self.y = y.copy()  # Store the endogenous variable
        self.create_intercept = create_intercept
        self.regression_type = regression_type
        self.results = {}

        if self.create_intercept:
            self.add_intercept()

    def add_intercept(self):
        # Adds a column of ones to x as the intercept
        self.x['intercept'] = 1

    def ols_regression(self):
        # Convert the x DataFrame to a NumPy matrix
        X = self.x.values
        X = np.array(X, dtype=float)

        # Ensure y is a 2D column vector
        y = np.array(self.y).reshape(-1, 1)

        # Performs ordinary least squares regression
        XtX = np.dot(X.T, X)  # X'X
        XtY = np.dot(X.T, self.y)  # X'y
        beta_hat = sp.linalg.solve(XtX, XtY, assume_a='pos')

        # Predicted values and residuals
        y_hat = np.dot(X, beta_hat)
        residuals = self.y - y_hat

        # Calculate degrees of freedom and residual variance
        n = len(self.y)  # number of rows
        k = X.shape[1]  # number of columns
        df = n - k
        residual_variance = np.sum(residuals**2) / df

        # Standard errors
        var_beta_hat = residual_variance * sp.linalg.inv(XtX)  # Variance-covariance matrix of beta_hat
        standard_errors = np.sqrt(np.diag(var_beta_hat))  # Standard errors

        t_stats = beta_hat / standard_errors
        p_values = sp.stats.t.sf(np.abs(t_stats), df)

        # Store the regression results in a dictionary
        variable_names = self.x.columns
        for i, var in enumerate(variable_names):
            self.results[var] = {
                "coefficient": beta_hat[i],
                "standard_error": standard_errors[i],
                "t_stat": t_stats[i],
                "p_value": p_values[i]
            }

    def logistic_regression(self, learning_rate=0.001, max_iter=10000, tol=1e-5, regularization=1e-6):
        # Performs logistic regression using gradient descent
        X = np.array(self.x, dtype=float)
        y = np.array(self.y).reshape(-1, 1)
        n, k = X.shape

        # Initialize beta coefficients
        beta = np.zeros((k, 1))

        def sigmoid(z):
            return 1 / (1 + np.exp(-z))

        def log_likelihood(beta):
            z = np.dot(X, beta)
            return np.sum(y * z - np.log(1 + np.exp(z)))

        previous_log_likelihood = -np.inf

        for _ in range(max_iter):
            z = np.dot(X, beta)
            predictions = sigmoid(z)
            gradient = np.dot(X.T, (y - predictions))
            beta_new = beta + learning_rate * gradient

            current_log_likelihood = log_likelihood(beta_new)
            if np.abs(current_log_likelihood - previous_log_likelihood) < tol:
                break
            beta = beta_new
            previous_log_likelihood = current_log_likelihood

        # Store coefficients as log odds-ratios (renamed to "coefficient" for compatibility)
        beta_hat = beta.flatten()


        # Regularized Hessian to ensure invertibility
        predictions_flat = predictions.flatten()
        hessian = -np.dot(X.T, X * (predictions_flat * (1 - predictions_flat)).reshape(-1, 1))
        hessian += np.eye(k) * regularization  # Regularization to stabilize inversion

        try:
          var_beta_hat = np.linalg.inv(hessian)
        except np.linalg.LinAlgError:
          raise ValueError("Hessian matrix is singular and cannot be inverted. Try adjusting regularization.")

        standard_errors = np.sqrt(np.diag(var_beta_hat))
        z_stats = beta_hat / standard_errors
        p_values = sp.stats.norm.sf(np.abs(z_stats))

        variable_names = self.x.columns
        for i, var in enumerate(variable_names):
            self.results[var] = {
                "coefficient": beta_hat[i],
                "standard_error": standard_errors[i],
                "z_stat": z_stats[i],
                "p_value": p_values[i]
            }

    def fit_model(self):
        # Determines the type of regression to run based on regression_type attribute
        if self.regression_type == "ols":
            self.ols_regression()
        elif self.regression_type == "logit":
            self.logistic_regression()
        else:
            raise ValueError("Invalid regression type. Choose 'ols' or 'logit'.")

    def summary(self):
        # Prints a summary table of regression results
        if self.regression_type == "ols":
            print(f"{'Variable name':<15}{'Coefficient':<15}{'Standard Error':<15}{'t-statistic':<15}{'p-value':<15}")
            print("=" * 75)
            for var, metrics in self.results.items():
                print(f"{var:<15}{metrics['coefficient']:<15.4f}{metrics['standard_error']:<15.4f}"
                      f"{metrics['t_stat']:<15.4f}{metrics['p_value']:<15.4f}")
        elif self.regression_type == "logit":
            print(f"{'Variable name':<15}{'Coefficient':<15}{'Standard Error':<15}{'z-statistic':<15}{'p-value':<15}")
            print("=" * 75)
            for var, metrics in self.results.items():
                print(f"{var:<15}{metrics['coefficient']:<15.4f}{metrics['standard_error']:<15.4f}"
                      f"{metrics['z_stat']:<15.4f}{metrics['p_value']:<15.4f}")